In [42]:
import os
from os.path import join as opj
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from nltools.stats import regress, zscore
from nltools.data import Brain_Data, Design_Matrix
from nltools.stats import find_spikes 
from nltools.mask import expand_mask
from nipype.interfaces.afni import Calc, TStat
import nibabel as nib

def make_motion_covariates(mc, tr):
    z_mc = zscore(mc)
    all_mc = pd.concat([z_mc, z_mc**2, z_mc.diff(), z_mc.diff()**2], axis=1)
    all_mc.fillna(value=0, inplace=True)
    return Design_Matrix(all_mc, sampling_freq=1/tr)

In [3]:
base_dir = '/projects/kuhl_lab/wanjiag/MONSTERA/derivatives/preprocess'
fmriprerp_dir = '/projects/kuhl_lab/wanjiag/MONSTERA/derivatives/fmriprep'
n_trunc = 6

In [4]:
tr = 1
outlier_cutoff = 3
subs = ['sub-MONSTERA01', 'sub-MONSTERA02', 'sub-MONSTERA03', 'sub-MONSTERA04',
        'sub-MONSTERA05', 'sub-MONSTERA06', 'sub-MONSTERA07']

for sub in subs:
    file_list = [x for x in glob.glob(opj(base_dir, sub, '*_space-T1w_desc-preproc_bold_trim6TRs_centered-masked*'))] 


In [5]:
file_list.sort()
file_list

['/projects/kuhl_lab/wanjiag/MONSTERA/derivatives/preprocess/sub-MONSTERA07/sub-MONSTERA07_task-01_space-T1w_desc-preproc_bold_trim6TRs_centered-masked.nii.gz',
 '/projects/kuhl_lab/wanjiag/MONSTERA/derivatives/preprocess/sub-MONSTERA07/sub-MONSTERA07_task-02_space-T1w_desc-preproc_bold_trim6TRs_centered-masked.nii.gz',
 '/projects/kuhl_lab/wanjiag/MONSTERA/derivatives/preprocess/sub-MONSTERA07/sub-MONSTERA07_task-03_space-T1w_desc-preproc_bold_trim6TRs_centered-masked.nii.gz',
 '/projects/kuhl_lab/wanjiag/MONSTERA/derivatives/preprocess/sub-MONSTERA07/sub-MONSTERA07_task-04_space-T1w_desc-preproc_bold_trim6TRs_centered-masked.nii.gz',
 '/projects/kuhl_lab/wanjiag/MONSTERA/derivatives/preprocess/sub-MONSTERA07/sub-MONSTERA07_task-05_space-T1w_desc-preproc_bold_trim6TRs_centered-masked.nii.gz',
 '/projects/kuhl_lab/wanjiag/MONSTERA/derivatives/preprocess/sub-MONSTERA07/sub-MONSTERA07_task-06_space-T1w_desc-preproc_bold_trim6TRs_centered-masked.nii.gz',
 '/projects/kuhl_lab/wanjiag/MONST

In [6]:
f = file_list[0]

In [7]:
f

'/projects/kuhl_lab/wanjiag/MONSTERA/derivatives/preprocess/sub-MONSTERA07/sub-MONSTERA07_task-01_space-T1w_desc-preproc_bold_trim6TRs_centered-masked.nii.gz'

In [10]:
derivative_dir = '/projects/kuhl_lab/wanjiag/MONSTERA/derivatives/'

out_dir = opj(derivative_dir, 'preprocess/%s' % (sub))

In [11]:
mask_output = opj(out_dir, '%s_space-T1w_desc-brain_intersect_mask.nii.gz' % (sub))

In [12]:
mask_output

'/projects/kuhl_lab/wanjiag/MONSTERA/derivatives/preprocess/sub-MONSTERA07/sub-MONSTERA07_space-T1w_desc-brain_intersect_mask.nii.gz'

In [13]:
run = os.path.basename(f).split('_')[1]

In [15]:
from nilearn.input_data import NiftiMasker

In [18]:
filtcutoff=128 #high-pass filter
print('now on session:', run)
epi_masker= NiftiMasker(mask_img=mask_output,  high_pass=1/filtcutoff, #high pass filter
    standardize=True,  # Are you going to zscore the data across time? 
    t_r=tr, 
    memory='nilearn_cache',  # Caches the mask in the directory given as a string here so that it is easier to load and retrieve
    memory_level=1,  # How much memory will you cache?
    verbose=0)

now on session: task-01


In [28]:
# load data and regress out confounds
epi_file=opj(out_dir, '%s_%s_space-T1w_desc-preproc_bold_trim%dTRs_centered-masked.nii.gz' % (sub, run, n_trunc))

In [29]:
covariates = pd.read_csv(glob.glob(os.path.join(fmriprerp_dir, sub, 'func', f'*{run}*tsv'))[0], sep='\t')

In [33]:
data = Brain_Data(epi_file)

AttributeError: 'str' object has no attribute 'find_spikes'

In [34]:
spikes = data.find_spikes(global_spike_cutoff=outlier_cutoff, diff_spike_cutoff=outlier_cutoff)

In [35]:
spikes

,TR,global_spike1,global_spike2,diff_spike1,diff_spike2,diff_spike3,diff_spike4
0,1,0,0,0,0,0,0
1,2,0,0,0,0,0,0
2,3,0,0,0,0,0,0
3,4,0,0,0,0,0,0
4,5,0,0,0,0,0,0
...,...,...,...,...,...,...,...
447,448,0,0,0,0,0,0
448,449,0,0,0,0,0,0
449,450,0,0,0,0,0,0
450,451,0,0,0,0,0,0


In [36]:
mc = covariates[['trans_x','trans_y','trans_z','rot_x', 'rot_y', 'rot_z']]
mc_cov = make_motion_covariates(mc, tr)
csf = covariates['csf'] 
fd = covariates['framewise_displacement']
dm = Design_Matrix(pd.concat([csf, fd, mc_cov, spikes.drop(labels='TR', axis=1)], axis=1), sampling_freq=1/tr)
dm = dm.add_poly(order=2, include_lower=True) # Add Intercept, Linear and Quadratic Trends
dm_trim = dm.iloc[n_trunc: , :]

In [37]:
epi_mask_data = epi_masker.fit_transform(epi_file,confounds=dm_trim)

In [38]:
out_dir

'/projects/kuhl_lab/wanjiag/MONSTERA/derivatives/preprocess/sub-MONSTERA07'

In [47]:
output_name = opj(out_dir,'test_%s_%s_desc-preproc_bold_trim%d_norm.nii.gz' % (sub, run, n_trunc))

In [48]:
output_name

'/projects/kuhl_lab/wanjiag/MONSTERA/derivatives/preprocess/sub-MONSTERA07/test_sub-MONSTERA07_task-01_desc-preproc_bold_trim6_norm.nii.gz'

In [44]:
avg_mask = nib.load(mask_output)
affine_mat = avg_mask.affine #should be the same as the epi data
print(avg_mask.shape)
coords = np.where(avg_mask.get_fdata())
bold_vol=[]
bold_vol=np.zeros((avg_mask.shape[0], avg_mask.shape[1], avg_mask.shape[2], epi_mask_data.shape[0]))
bold_vol[coords[0], coords[1], coords[2], :] = epi_mask_data.T
print('epi_mask_data shape:', bold_vol.shape)

(86, 105, 80)
epi_mask_data shape: (86, 105, 80, 452)


In [46]:
bold_nii = nib.Nifti1Image(bold_vol, affine_mat)
#hdr = bold_nii.header  # get a handle for the .nii file's header
#hdr.set_zooms((orig_dimsize[0], orig_dimsize[1], orig_dimsize[2], orig_dimsize[3]))
nib.save(bold_nii, output_name)

In [ ]:
bold_nii = nib.Nifti1Image(bold_vol, affine_mat)
hdr = bold_nii.header  # get a handle for the .nii file's header
hdr.set_zooms((orig_dimsize[0], orig_dimsize[1], orig_dimsize[2], orig_dimsize[3]))
nib.save(bold_nii, output_name)

In [24]:
#sub = os.path.basename(f).split('_')[0]
# sub

In [26]:
data = Brain_Data(f)
#smoothed = data
#smoothed = data.smooth(fwhm=fwhm)

In [27]:
spikes = data.find_spikes(global_spike_cutoff=outlier_cutoff, diff_spike_cutoff=outlier_cutoff)


In [30]:
covariates = pd.read_csv(glob.glob(os.path.join(fmriprerp_dir, sub, 'func', '*tsv'))[0], sep='\t')

In [31]:
covariates.framewise_displacement

0           NaN
1      0.164173
2      0.199301
3      0.085888
4      0.178369
         ...   
453    0.155007
454    0.055026
455    0.122061
456    0.070154
457    0.187903
Name: framewise_displacement, Length: 458, dtype: float64

In [32]:
mc = covariates[['trans_x','trans_y','trans_z','rot_x', 'rot_y', 'rot_z']]

In [33]:
mc_cov = make_motion_covariates(mc, tr)

In [34]:
mc_cov

,trans_x,trans_y,trans_z,rot_x,rot_y,rot_z,trans_x,trans_y,trans_z,rot_x,...,trans_z,rot_x,rot_y,rot_z,trans_x,trans_y,trans_z,rot_x,rot_y,rot_z
0,0.570403,0.193032,-1.279040,1.798006,1.771041,0.843136,0.325360,0.037261,1.635943,3.232824,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.604833,0.980696,-0.507892,1.339015,2.332046,0.971070,0.365823,0.961765,0.257954,1.792962,...,0.771148,-0.458990,0.561005,0.127934,0.001185,0.620415,0.594669,0.210672,0.314726,0.016367
2,0.434855,-1.023800,-0.051321,1.464263,1.802434,0.843136,0.189099,1.048167,0.002634,2.144067,...,0.456571,0.125248,-0.529612,-0.127934,0.028892,4.018007,0.208457,0.015687,0.280489,0.016367
3,0.616575,-0.477767,0.030806,1.601442,2.141941,0.956156,0.380165,0.228261,0.000949,2.564617,...,0.082127,0.137179,0.339507,0.113020,0.033022,0.298153,0.006745,0.018818,0.115265,0.012773
4,0.658560,1.189665,-0.390477,1.259424,2.054529,1.098380,0.433701,1.415304,0.152472,1.586149,...,-0.421283,-0.342018,-0.087412,0.142224,0.001763,2.780330,0.177479,0.116976,0.007641,0.020228
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
453,-0.655401,0.509222,2.567503,0.227274,1.602191,-0.457527,0.429550,0.259307,6.592073,0.051653,...,-0.248749,0.347832,0.269281,0.063710,0.007980,2.067097,0.061876,0.120987,0.072512,0.004059
454,-0.534354,0.375747,2.642604,0.456274,2.071035,-0.457527,0.285535,0.141186,6.983358,0.208186,...,0.075101,0.229000,0.468844,0.000000,0.014652,0.017816,0.005640,0.052441,0.219815,0.000000
455,-0.481063,1.177201,3.196049,0.108814,1.986202,-0.401842,0.231421,1.385802,10.214730,0.011841,...,0.553445,-0.347460,-0.084834,0.055685,0.002840,0.642328,0.306301,0.120728,0.007197,0.003101
456,-0.529495,1.662946,2.718198,0.104320,2.406287,-0.412002,0.280365,2.765388,7.388602,0.010883,...,-0.477851,-0.004494,0.420085,-0.010160,0.002346,0.235948,0.228342,0.000020,0.176471,0.000103


In [35]:
csf = covariates['csf'] 
fd = covariates['framewise_displacement']

In [36]:
dm = Design_Matrix(pd.concat([csf, fd, mc_cov, spikes.drop(labels='TR', axis=1)], axis=1), sampling_freq=1/tr)


In [37]:
dm = dm.add_poly(order=2, include_lower=True) # Add Intercept, Linear and Quadratic Trends

In [40]:
dm_trim = dm.iloc[n_trunc: , :]

In [45]:
data.X = dm_trim

In [46]:
stats = data.regress()

In [31]:
#stats['residual'].data = np.float32(stats['residual'].data) # cast as float32 to reduce storage space

In [47]:
output_dir = os.path.join('/projects/kuhl_lab/wanjiag/MONSTERA/derivatives/preprocess/', sub)

In [58]:
denoise_output = os.path.join(output_dir, f'{sub}_{task_name}_space-T1w_desc-preproc_bold_denoise_no-smooth.nii.gz')

In [56]:
stats['residual'].write(denoise_output)

In [59]:
denoise_output

'/projects/kuhl_lab/wanjiag/MONSTERA/derivatives/preprocess/sub-MONSTERA07/sub-MONSTERA07_task-01_space-T1w_desc-preproc_bold_denoise_no-smooth.nii.gz'

## z-scores

In [63]:
# Center values
## Calculate mean images
print('calculating mean......')
mean_output = opj(output_dir, '%s_%s_space-T1w_desc-preproc_bold_bold_denoise_no-smooth_mean-img.nii.gz' % (sub, task_name))
tstat = TStat()
tstat.inputs.in_file = denoise_output
tstat.inputs.args = '-mean'
tstat.inputs.out_file = mean_output
print(tstat.cmdline)
res = tstat.run() 
print('calculating mean...output to %s'%(mean_output))

calculating mean......
3dTstat -mean -prefix /projects/kuhl_lab/wanjiag/MONSTERA/derivatives/preprocess/sub-MONSTERA07/sub-MONSTERA07_task-01_space-T1w_desc-preproc_bold_bold_denoise_no-smooth_mean-img.nii.gz /projects/kuhl_lab/wanjiag/MONSTERA/derivatives/preprocess/sub-MONSTERA07/sub-MONSTERA07_task-01_space-T1w_desc-preproc_bold_denoise_no-smooth.nii.gz
220426-11:46:00,983 nipype.interface INFO:
	 stderr 2022-04-26T11:46:00.983656:++ 3dTstat: AFNI version=AFNI_18.2.04 (Jul  6 2018) [64-bit]
220426-11:46:00,991 nipype.interface INFO:
	 stderr 2022-04-26T11:46:00.983656:++ Authored by: KR Hammett & RW Cox
220426-11:46:01,39 nipype.interface INFO:
	 stderr 2022-04-26T11:46:01.039833:** AFNI converts NIFTI_datatype=64 (FLOAT64) in file /projects/kuhl_lab/wanjiag/MONSTERA/derivatives/preprocess/sub-MONSTERA07/sub-MONSTERA07_task-01_space-T1w_desc-preproc_bold_denoise_no-smooth.nii.gz to FLOAT32
220426-11:46:01,42 nipype.interface INFO:
	 stderr 2022-04-26T11:46:01.039833:     Warnings of

In [64]:
## Calculate SD images
print('calculating SD......')
sd_output = opj(output_dir, '%s_%s_space-T1w_desc-preproc_bold_bold_denoise_no-smooth_sd-img.nii.gz' % (sub, task_name))
tstat = TStat()
tstat.inputs.in_file = denoise_output
tstat.inputs.args = '-stdevNOD'
tstat.inputs.out_file = sd_output
print(tstat.cmdline)
res = tstat.run() 
print('calculating SD...output to %s'%(sd_output))

calculating SD......
3dTstat -stdevNOD -prefix /projects/kuhl_lab/wanjiag/MONSTERA/derivatives/preprocess/sub-MONSTERA07/sub-MONSTERA07_task-01_space-T1w_desc-preproc_bold_bold_denoise_no-smooth_sd-img.nii.gz /projects/kuhl_lab/wanjiag/MONSTERA/derivatives/preprocess/sub-MONSTERA07/sub-MONSTERA07_task-01_space-T1w_desc-preproc_bold_denoise_no-smooth.nii.gz
220426-11:47:14,414 nipype.interface INFO:
	 stderr 2022-04-26T11:47:14.413897:++ 3dTstat: AFNI version=AFNI_18.2.04 (Jul  6 2018) [64-bit]
220426-11:47:14,420 nipype.interface INFO:
	 stderr 2022-04-26T11:47:14.413897:++ Authored by: KR Hammett & RW Cox
220426-11:47:14,422 nipype.interface INFO:
	 stderr 2022-04-26T11:47:14.413897:** AFNI converts NIFTI_datatype=64 (FLOAT64) in file /projects/kuhl_lab/wanjiag/MONSTERA/derivatives/preprocess/sub-MONSTERA07/sub-MONSTERA07_task-01_space-T1w_desc-preproc_bold_denoise_no-smooth.nii.gz to FLOAT32
220426-11:47:14,427 nipype.interface INFO:
	 stderr 2022-04-26T11:47:14.413897:     Warnings 

In [65]:
## Using AFNI to calculate z-scores
print('Zscore-ing data ......')
calc = Calc()
calc.inputs.in_file_a = denoise_output
calc.inputs.in_file_b = mean_output
calc.inputs.in_file_c = sd_output

calc.inputs.expr= '(a-b)/c'
out_file = opj(output_dir, '%s_%s_space-T1w_desc-preproc_bold_bold_denoise_no-smooth_z-score-img.nii.gz' % (sub, task_name))
calc.inputs.out_file = out_file

print(calc.cmdline)
calc.run()
print('Centering data...output to %s'%(out_file))

Zscore-ing data ......
3dcalc -a /projects/kuhl_lab/wanjiag/MONSTERA/derivatives/preprocess/sub-MONSTERA07/sub-MONSTERA07_task-01_space-T1w_desc-preproc_bold_denoise_no-smooth.nii.gz -b /projects/kuhl_lab/wanjiag/MONSTERA/derivatives/preprocess/sub-MONSTERA07/sub-MONSTERA07_task-01_space-T1w_desc-preproc_bold_bold_denoise_no-smooth_mean-img.nii.gz -c /projects/kuhl_lab/wanjiag/MONSTERA/derivatives/preprocess/sub-MONSTERA07/sub-MONSTERA07_task-01_space-T1w_desc-preproc_bold_bold_denoise_no-smooth_sd-img.nii.gz -expr "(a-b)/c" -prefix /projects/kuhl_lab/wanjiag/MONSTERA/derivatives/preprocess/sub-MONSTERA07/sub-MONSTERA07_task-01_space-T1w_desc-preproc_bold_bold_denoise_no-smooth_z-score-img.nii.gz
220426-11:49:38,419 nipype.interface INFO:
	 stderr 2022-04-26T11:49:38.418894:++ 3dcalc: AFNI version=AFNI_18.2.04 (Jul  6 2018) [64-bit]
220426-11:49:38,425 nipype.interface INFO:
	 stderr 2022-04-26T11:49:38.418894:++ Authored by: A cast of thousands
220426-11:49:38,427 nipype.interface INF

NameError: name 'opj' is not defined

In [ ]:
epi_mask_data_all=np.vstack([epi_mask_data_all,epi_mask_data])
nTR_all=np.vstack([nTR_all,epi_mask_data.shape[0]])